In [387]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.model_selection import cross_val_score
from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
import numpy as np
import math
import statistics
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from clean import clean_data
from output import outputs
from tests import model_RMSE, pipeline, test_features, xyz_test

In [328]:
train = pd.read_csv('data/train.csv')
predict = pd.read_csv('data/predict.csv')
sample = pd.read_csv('data/sample_submission.csv')

# Testing models with different features

In [330]:
best_features=test_features(df,10)

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1086449619.3681412, tolerance: 52508017.77129175
  model = cd_fast.enet_coordinate_descent(


In [331]:
best_features

,all_f,no_xyz,no_depth,no_depth_no_xyz
<class 'sklearn.linear_model._base.LinearRegression'>,1223.746736,1245.947298,1212.241536,1247.862938
<class 'sklearn.tree._classes.DecisionTreeRegressor'>,748.797380,737.222073,751.980063,722.152664
<class 'sklearn.linear_model._coordinate_descent.ElasticNet'>,1751.330318,2990.254451,1752.766533,2975.609944
<class 'sklearn.linear_model._ridge.Ridge'>,1212.269503,1240.652748,1230.180921,1228.450356
<class 'sklearn.linear_model._coordinate_descent.Lasso'>,1243.969716,1246.707746,1222.797854,1234.688558
<class 'sklearn.neighbors._regression.KNeighborsRegressor'>,900.756086,1889.690101,821.632358,1449.201132
<class 'sklearn.ensemble._forest.RandomForestRegressor'>,555.816530,560.061387,553.688108,581.471348


Here I can decide the top 3 model with the corresponding top 2 features.

The ranking model is as follows:

1. RandomForestRegressor: no_depht, all_f
2. DecisionTreeRegressor: no_depth_no_xyz, no_xyz
3. KNeighborsRegressor: no_depth, all_f

# 1. RandomForestRegressor: no_depht, all_f

In [379]:
df = pd.read_csv('data/train.csv').drop(columns=['id'])
df_no_depth=df.drop(columns=['depth'])
rfr_all=pipeline(df, RandomForestRegressor)
rfr_no_depth=pipeline(df_no_depth, RandomForestRegressor)

In [380]:
rfr_all

,cross_val_score_mean,RMSE,R2
0,-300491.763044,563.138744,0.980574


In [381]:
rfr_no_depth

,cross_val_score_mean,RMSE,R2
0,-314782.663085,555.635497,0.980288


# 2. DecisionTreeRegressor: no_depth_no_xyz, no_xyz

In [378]:
df = pd.read_csv('data/train.csv').drop(columns=['id'])
df_no_depth_no_xyz=df.drop(columns=['depth','x','y','z'])
df_no_xyz=df.drop(columns=['x','y','z'])
dtr_no_depth_xyz=pipeline(df_no_depth_no_xyz, DecisionTreeRegressor)
dtr_no_xyz=pipeline(df_no_xyz, DecisionTreeRegressor)

In [382]:
dtr_no_depth_xyz

,cross_val_score_mean,RMSE,R2
0,-550363.949213,743.064264,0.96708


In [383]:
dtr_no_xyz

,cross_val_score_mean,RMSE,R2
0,-524447.569199,774.11532,0.962581


# 3. KNeighborsRegressor: no_depth, all_f

In [384]:
df = pd.read_csv('data/train.csv').drop(columns=['id'])
df_no_depth=df.drop(columns=['depth'])
knr_no_depth=pipeline(df_no_depth, KNeighborsRegressor)
knr_all=pipeline(df, KNeighborsRegressor)

In [385]:
knr_no_depth

,cross_val_score_mean,RMSE,R2
0,-686884.547718,817.374042,0.958703


In [386]:
knr_all

,cross_val_score_mean,RMSE,R2
0,-808602.086849,896.041759,0.950545


# Prediction to csv

In [391]:
train = pd.read_csv('data/train.csv')
predict = pd.read_csv('data/predict.csv')

In [392]:
outputs(RandomForestRegressor,predict,train,'outputs/prediction')

# Extras

In [ ]:
# Before defining the function test_features I made the tests one by one, SGDRegressor and SVR are too long process 
# for a very bad result, so I leave them printed here, but I add them to the final function:
# I definde this function in order to decide if I should drop x,y,z or not:
def xyz_test(df,df_drop_xyz,model):
    xyz=[]
    no_xyz=[]
    for i in range(100):
        xyz.append(model_RMSE(df,model))
        no_xyz.append(model_RMSE(df_drop_xyz,model))   
        return {"xyz":statistics.mean(xyz), "no_xyz":statistics.mean(no_xyz)}

In [225]:
df = pd.read_csv('data/train.csv').drop(columns=['id'])
df_drop_xyz=df.drop(columns=['x','y','z'])
xyz_test(df,df_drop_xyz,SVR)

{'xyz': 4164.9956905341505, 'no_xyz': 4320.70844077423}

In [247]:
df = pd.read_csv('data/train.csv').drop(columns=['id'])
df_drop_xyz=df.drop(columns=['x','y','z'])
xyz_test(df,df_drop_xyz,SGDRegressor)

{'xyz': 77140220.93606305, 'no_xyz': 114719750.69685547}

# GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

y = df['price']
x = df.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
model = SVR()
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)
GridSearchCV(estimator=SVR(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})
sorted(clf.cv_results_.keys())

In [179]:
pd.DataFrame.from_dict(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,17.847624,0.367658,1.504411,0.076706,1,linear,"{'C': 1, 'kernel': 'linear'}",0.806575,0.795566,0.798468,0.798293,0.796857,0.799152,0.003859,2
1,30.925639,0.572925,3.848920,0.043687,1,rbf,"{'C': 1, 'kernel': 'rbf'}",-0.131857,-0.147335,-0.132104,-0.129266,-0.129897,-0.134092,0.006711,4
2,20.184431,0.916863,1.528313,0.070182,10,linear,"{'C': 10, 'kernel': 'linear'}",0.881503,0.868582,0.875496,0.870587,0.870352,0.873304,0.004699,1
3,31.141748,0.379140,3.872318,0.079626,10,rbf,"{'C': 10, 'kernel': 'rbf'}",-0.010597,-0.028384,-0.012403,-0.012140,-0.012248,-0.015154,0.006647,3
